In [6]:

# -*- coding:utf-8 -*-
 
##随机挑选部分内容
# encoding:utf-8
import random
from random import randint
import json
 
oldf = open('Comments.json', 'r', encoding="UTF-8")
lines = oldf.readlines()
oldf.close()


"\nnewf = open('sun.txt', 'w')   ###挑选400行\nn = 0\nresultList = random.sample(range(0, 200000), 5000)  # sample(x,y)函数的作用是从序列x中，随机选择y个不重复的元素。\n \n\nfor i in resultList:\n    newf.write(lines[i])\noldf.close()\nnewf.close()\n"

In [7]:
print(len(lines))

246239


In [43]:
newf = open('sun.txt', 'w', encoding='UTF-8') 

n = 0
for i in range(len(lines)):
    if "content" not in lines[i]:
        continue
    json_data = json.loads(lines[i])
    if json_data["content"] == "":
        continue
    newf.write(json_data["content"]+'\n')
newf.close()

In [2]:
import jieba
import re


def tokenize(text):
    """
    带有语料清洗功能的分词函数
    """
    text = re.sub("\{%.+?%\}", " ", text)           # 去除 {%xxx%} (地理定位, 微博话题等)
    text = re.sub("回复@\S+:", '', text)            # 去除回复@
    text = re.sub("@.+?( |$)", " ", text)           # 去除 @xxx (用户名)
    
    text = re.sub("【.+?】", " ", text)              # 去除 【xx】 (里面的内容通常都不是用户自己写的)
    icons = re.findall("\[.+?\]", text)             # 提取出所有表情图标
    text = re.sub("\[.+?\]", "IconMark", text)      # 将文本中的图标替换为`IconMark`
    
    tokens = []
    for k, w in enumerate(jieba.lcut(text)):
        w = w.strip()
        if "IconMark" in w:                         # 将IconMark替换为原图标
            for i in range(w.count("IconMark")):
                tokens.append(icons.pop(0))
        elif w and w != '\u200b' and w.isalpha():   # 只保留有效文本
                tokens.append(w)
    return tokens


def load_corpus(path):
    """
    加载语料库
    """
    data = []
    with open(path, "r", encoding="utf8") as f:
        for line in f:
            content = tokenize(line)             # 分词
            if content!= []:
                data.append((content))
    return data

In [48]:
tokenize("你好")

['你好']

In [55]:
sun_data

[['沙发'],
 ['么', '么', '哒', '大', '某人'],
 ['互粉', '一个', '孙董'],
 ['微', '博爱'],
 ['图片', '评论', '评论', '配图'],
 ['大', '老师', '这', '肚子', '几个', '月', '了'],
 ['大', '师兄', '粉', '我', 'o'],
 ['图片', '评论', '评论', '配图'],
 ['孙董', '求个', 'LOL', '好友', '位', '啊'],
 ['我', '是', '护疮宝'],
 ['李', '某人', '滚出', '我', '孙', '董事长', '的', '直播间'],
 ['恭喜', '大', '某人', '成立', '痔疮', '工作室'],
 ['日死', '你', '的', '妈'],
 ['大', '师兄', '你', '肚子', '又', '大', '了', '肯定', '是', '过年', '吃', '太', '好'],
 ['日死', '你', '的', '妈', '老子', '要', '看', 'gta', '看', '逃生'],
 ['有事没事', '发个', '动态', '知道', '没', '日死', '你', '的', '吗'],
 ['日死', '你', '的', '妈', '还', '不来波', '网恋', '教学'],
 ['嗨', '呀', '教父', '网恋', '教学', '走', '一波'],
 ['抽象', '工作室', '第一', '傻狗'],
 ['仅靠', '肚子', '就', '能', '挺', '破', 'DXX', '的', '批'],
 ['嗨', '呀', '[微笑]'],
 ['秋秋爱'],
 ['孙笑川', '我', '日', '死', '你', '的', '吗'],
 ['我', '宣布', '你', '被', '开除', '了'],
 ['嗨', '呀', '我', '是', '护', '窗宝'],
 ['嗨', '呀', '评论', '配图'],
 ['大', '师兄', '你', '司马', '带六区', '的', '节奏'],
 ['嗨', '呀', '日死', '你', '的', '吗'],
 ['日死', '你', '的', '吗', '痔疮', '怪'],
 

In [49]:
import pandas as pd
sun_data = load_corpus("sun.txt")

ValueError: 1 columns passed, passed data had 125 columns

In [46]:
df.head()

NameError: name 'df' is not defined

In [50]:
sun_data = [t for t in sun_data if t!=[]]

In [52]:
len(sun_data)

229380

In [53]:
stopwords = []
with open("stopwords.txt", "r", encoding="utf8") as f:
    for w in f:
        stopwords.append(w.strip())

In [54]:
# 训练词向量
from gensim.models import FastText
model = FastText(sun_data, 
                 size=100,
                 window=5, 
                 min_count=3, # 只保留出现次数大于3的词语
                 iter=1000,  # 10000次训练
                 min_n=2,     # 默认为3,因为文本是中文这里改为2
                 max_n=4,     # 默认为6,因为文本是中文这里改为5
                 word_ngrams=1)

KeyboardInterrupt: 

In [ ]:
model.save("model/model_sun.txt")

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
data_str = [" ".join(content) for content in sun_data]
tfidf = TfidfVectorizer(token_pattern='\[?\w+\]?', stop_words=stopwords)
tfidf_fit = tfidf.fit_transform(data_str)

C:\Users\lenovo\Anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['元', '吨', '数', '末'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [42]:
from gensim.models import FastText
model = FastText.load("model/model_100.txt")

In [1]:
sun_data = [[1,2],[2,3]]

In [2]:
sun_data = [(t, ) for t in sun_data]

In [3]:
sun_data

[([1, 2],), ([2, 3],)]

In [6]:
import random
sun_data = random.sample(sun_data, 1)

In [7]:
sun_data

[([1, 2],)]

In [8]:
def tokenize(text):
    """
    带有语料清洗功能的分词函数
    """
    text = re.sub("\{%.+?%\}", " ", text)           # 去除 {%xxx%} (地理定位, 微博话题等)
    text = re.sub("回复@\S+:", '', text)            # 去除回复@
    text = re.sub("@.+?( |$)", " ", text)           # 去除 @xxx (用户名)
    
    text = re.sub("【.+?】", " ", text)              # 去除 【xx】 (里面的内容通常都不是用户自己写的)
    icons = re.findall("\[.+?\]", text)             # 提取出所有表情图标
    text = re.sub("\[.+?\]", "IconMark", text)      # 将文本中的图标替换为`IconMark`

    tokens = []
    for k, w in enumerate(jieba.lcut(text)):
        w = w.strip()
        if "IconMark" in w:                         # 将IconMark替换为原图标
            for i in range(w.count("IconMark")):
                tokens.append(icons.pop(0))
        elif w and w != '\u200b' and w.isalpha():   # 只保留有效文本
                tokens.append(w)
    return tokens

In [ ]:
def load_corpus_2(path):
    """
    加载语料库
    """
    data = []
    with open(path, "r", encoding="utf8") as f:
        for line in f:
            [_, seniment, content] = line.split(",", 2)
            content = tokenize(content)             # 分词
            if content != []:
                data.append((content, int(seniment)))
    return data

In [5]:
l = [('回复@吃薯片儿要多喝水:🈶️内鬼，所以你🔥不了\n', 0.0), ('回复@黑天鹅港的阿卜杜拉:砰砰砰，搞快丶\n', 1.0), ('回复@豆苏土x:图片评论 评论配图\n', 1.0), ('回复@看相解惑法师:带法师，最喜欢你传播封建迷信的样子，🌶真的🐮🍺，不要理会他们的谣言和中伤，[太开心]\n', 0.0), ('？？？这么快就结束了？？？？\n',  1.0), ('回复@王子公园的彩虹:给👴🏻 逗笑了\n', 1.0), ('回复@CherryrAind:就你好看行吗？\n', 0.0), ('回复@xiongzikaiiii:卖鸡儿大学？那真的牛批\n', 1.0), ('我在群里发黄图没意思退了\n', 0.0), ('回复@带带韩亚威:Gkd\n', 1.0), ('不要脸蹭人家热度！\n', 1.0), ('回复@带带大师兄:透你🐴 ，你就是刚刚追我的那个屠夫？你的勾子也太准了8⃣\n', 0.0), ('回复@天翼灬飘渺:gkd\n', 1.0), ('回复@观音桥毕加索:你再骂？\n', 1.0), ('回复@你是我许的第一个愿:你妈死扣\n', 1.0), ('回复@水度走刀广林友:你有点给力\n', 0.0), ('回复@李云浩改了个无聊的微博名字:你们这样成天消费老艺术家，并以此为乐，有意思吗？基本的素质呢？我也真是佛了，说到佛，我就想到了师徒四人西天取经，明年下半年，中美合拍的西游记即将正式开机，我继续扮演美猴王孙悟空，我会用美猴王艺术形象努力创造一个正能量的 形象，文体两开花，弘扬中华文化，希望大家能多多关注。\n', 1.0), ('回复@她咧:带哥私一哈\n', 1.0), ('你们is 为什么总要杀人？？？\n', 1.0), ('等着倒闭吧 ，铁憨憨\n', 1.0), ('无内鬼，发点🍺 站裸舞\n', 1.0), ('回复@你该gai备注:实力印象派画家\n', 0.0), ('孤儿招个亲妈乱伦？\n', 1.0), ('回复@Chigga_Ye:我只看到了四个字\n', 0.0), ('回复@杨鑫onewaygo:你个瓜娃儿\n', 1.0), ('暗示明晚练功？\n', 1.0), ('回复@躺着站军姿:求番号\n', 0.0), ('回复@尤里乌斯·西撒·齐贝林:搞快点\n', 1.0), ('回复@带带手冲鲲:你🐴biss\n', 0.0), ('回复@胡来的碧池:@艾owen 进来甩狙\n', 0.0), ('[蜘蛛侠]\n', 1.0), ('回复@开会司马脸:疤智一口一个太君叫的真欢[偷笑]\n', 0.0), ('还不更新？孙🐶再⑧更新咖喱人怎么样大家说了算嗷[太开心][太开心]\n', 1.0), ('回复@宫城种田:插眼\n', 1.0), ('回复@Mr脑洞君:真视守卫\n', 1.0), ('回复@为父_:带善人搞快点\n', 1.0), ('回复@DoubleS不是rapper:不狡辩[太开心]\n', 0.0), ('回复@GuowHy:直接参加你的灵堂就完事 了\n', 0.0), ('回复@葬爱家族徐少:你🔥 你🐴 呢¿加大力度#97  就完事了\n', 0.0), ('回复@寻梦豪biubiubiu:@STACEYii 这图应该不是p的\n', 0.0), ('笑到最后才是王.jpg\n', 1.0), ('你怎么还没死\n', 1.0), ('铁汁 莱莱好大哦🙊\n', 0.0), ('👴票都交完洛 gkd\n', 1.0), ('回复@未來福音extra:👴在地铁上👴不敢看无内鬼 ，👴真是个five\n', 1.0), ('回复@哈阿呦:车牌，秋梨膏\n', 1.0), ('回复@看相解惑法师:法师你🐴是他🐴琥珀🍺，大🔥都不想透了\n', 1.0), ('回复@奇迹有颜色吗:@03-23-07-27 这是👴现在最爱的环节[嘻嘻][嘻嘻]\n', 0.0), ('回复@女朋友的逼:蹭热度的滚出克\n', 1.0), ('回复@庄耀源同学:我是傲慢的演奏家 你萌呢？\n', 0.0)]

In [9]:
res = []
for text, p in l:
    if p==1:
        prediction = "正面"
    else:
        prediction = "反面"
    res.append((text.strip(), prediction))

In [10]:
res

[('回复@吃薯片儿要多喝水:🈶️内鬼，所以你🔥不了', '反面'),
 ('回复@黑天鹅港的阿卜杜拉:砰砰砰，搞快丶', '正面'),
 ('回复@豆苏土x:图片评论 评论配图', '正面'),
 ('回复@看相解惑法师:带法师，最喜欢你传播封建迷信的样子，🌶真的🐮🍺，不要理会他们的谣言和中伤，[太开心]', '反面'),
 ('？？？这么快就结束了？？？？', '正面'),
 ('回复@王子公园的彩虹:给👴🏻 逗笑了', '正面'),
 ('回复@CherryrAind:就你好看行吗？', '反面'),
 ('回复@xiongzikaiiii:卖鸡儿大学？那真的牛批', '正面'),
 ('我在群里发黄图没意思退了', '反面'),
 ('回复@带带韩亚威:Gkd', '正面'),
 ('不要脸蹭人家热度！', '正面'),
 ('回复@带带大师兄:透你🐴 ，你就是刚刚追我的那个屠夫？你的勾子也太准了8⃣', '反面'),
 ('回复@天翼灬飘渺:gkd', '正面'),
 ('回复@观音桥毕加索:你再骂？', '正面'),
 ('回复@你是我许的第一个愿:你妈死扣', '正面'),
 ('回复@水度走刀广林友:你有点给力', '反面'),
 ('回复@李云浩改了个无聊的微博名字:你们这样成天消费老艺术家，并以此为乐，有意思吗？基本的素质呢？我也真是佛了，说到佛，我就想到了师徒四人西天取经，明年下半年，中美合拍的西游记即将正式开机，我继续扮演美猴王孙悟空，我会用美猴王艺术形象努力创造一个正能量的 形象，文体两开花，弘扬中华文化，希望大家能多多关注。',
  '正面'),
 ('回复@她咧:带哥私一哈', '正面'),
 ('你们is 为什么总要杀人？？？', '正面'),
 ('等着倒闭吧 ，铁憨憨', '正面'),
 ('无内鬼，发点🍺 站裸舞', '正面'),
 ('回复@你该gai备注:实力印象派画家', '反面'),
 ('孤儿招个亲妈乱伦？', '正面'),
 ('回复@Chigga_Ye:我只看到了四个字', '反面'),
 ('回复@杨鑫onewaygo:你个瓜娃儿', '正面'),
 ('暗示明晚练功？', '正面'),
 ('回复@躺着站军姿:求番号', '反面'),
 ('回复@尤里乌斯·西撒·齐贝林:搞快点', '正面'),
 ('回复@带